In [1]:
import pandas as pd

In [3]:
train = pd.read_csv('funda_train.csv')

In [5]:
type(train['transacted_date'][0])

str

In [10]:
pd.to_datetime(train['transacted_date'])[0].weekday()

2

In [12]:
# apply : 이 컬럼에 있는 원소 하나하나에게 함수를 먹임
train['weekday'] = pd.to_datetime(train['transacted_date']).apply(lambda x: x.weekday())

In [13]:
train.head()

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,weekday
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857,2
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857,2
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000,2
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857,2
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000,3


In [31]:
train[(train['transacted_date'] >= '2018-03-01') & (train['transacted_date'] <= '2018-05-31')].groupby(['store_id'])['amount'].sum().to_csv('baseline.csv')

C:\Users\Minju\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [33]:
# reset_index() 를 해줘야 store_id column 이 생긴다. 그 전에는 index
baseline = train[(train['transacted_date'] >= '2018-03-01') & (train['transacted_date'] <= '2018-05-31')].groupby(['store_id'])['amount'].sum().reset_index()

In [34]:
baseline.head()

,store_id,amount
0,0,2.171000e+06
1,1,2.977143e+05
2,2,9.461429e+05
3,4,2.475643e+06
4,5,1.111857e+06


# 제출한 답변에 누락된 값이 없는지 확인

In [17]:
# nunique : count + unique
train['store_id'].nunique()

1967

In [35]:
baseline['store_id'].nunique()

1955

# 누락된 값 12개 찾기

In [44]:
train_store_id = list(train['store_id'].unique())

In [45]:
baseline_store_id = list(baseline['store_id'].unique())

## 풀이1 for loop

In [61]:
erased_store_id = []
for i in train_store_id:
    if i not in baseline_store_id:
        erased_store_id.append(i)
# print(erased_store_id)
erased_store_id

[271, 279, 515, 588, 706, 795, 999, 1379, 1435, 2081, 2094, 2119]

In [85]:
train271 = train[train['store_id'] == 271]
train271[(train271['transacted_date'] >= '2018-03-01') & (train271['transacted_date'] <= '2018-05-31')]

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,weekday


## 풀이2 boolean 부정형

In [92]:
# 앞에 ~ 넣으면 Boolean 부정형
train_missing = train[~train['store_id'].isin(baseline['store_id'])]

In [100]:
train_missing['store_id'].unique()

array([ 271,  279,  515,  588,  706,  795,  999, 1379, 1435, 2081, 2094,
       2119], dtype=int64)

In [94]:
train_missing[(train_missing['transacted_date'] >= '2018-03-01') & (train_missing['transacted_date'] <= '2018-05-31')]

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,weekday


## 풀이3 Set

In [98]:
# set 은 unique 한 값만 가져온다
set(train['store_id']) - set(baseline['store_id'])

{271, 279, 515, 588, 706, 795, 999, 1379, 1435, 2081, 2094, 2119}

# Series 와 DataFrame
- Series 는 컬럼이 한 줄
- DataFrame 은 컬럼이 여러줄

In [65]:
pd.Series([1, 2, 3, 4])

0    1
1    2
2    3
3    4
dtype: int64

In [72]:
# Series 는 index 를 지정해줄 수 있다.(Dictionary 처럼) 
# List 는 index 자동 지정
pd.Series([1, 2, 3, 4], index=['A', 'B', 'C', 'D'])

A    1
B    2
C    3
D    4
dtype: int64

In [75]:
# reset_index() 해주면 DataFrame 이 되면서 새로운 index 생성
# set_index('index') 해주면 다시 index 지정, still DataFrame
pd.Series([1, 2, 3, 4], index=['A', 'B', 'C', 'D']).reset_index().set_index('index')

,0
index,
A,1
B,2
C,3
D,4


In [64]:
pd.DataFrame([1, 2, 3, 4])

,0
0,1
1,2
2,3
3,4


In [ ]:
# 1. 2019년 1월부터 2월까지 매출의 합 / 59일 * 92일
# 2. 92일동안 월화수목금토일의 갯수를 구해서